## Predicting Flight Delays
### Exploratory Data Analysis
Team: Ali, Khaildyn, Mobola
## Task 4

In [2]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go

%matplotlib inline

In [3]:
# Read files
df_flights = pd.read_csv("data/flights.csv")

#### **Task 4**: How taxi times changing during the day? Does higher traffic lead to bigger taxi times?

In [5]:
taxi_df = df_flights.filter(['taxi_in', 'taxi_out', 'dep_time','arr_time'], axis=1)
taxi_df

,taxi_in,taxi_out,dep_time,arr_time
0,4.0,20.0,1453.0,1635.0
1,10.0,38.0,1757.0,2201.0
2,4.0,16.0,1630.0,1841.0
3,2.0,12.0,2114.0,2341.0
4,4.0,16.0,2015.0,2316.0
...,...,...,...,...
195867,8.0,27.0,1618.0,1826.0
195868,18.0,12.0,1918.0,2045.0
195869,2.0,22.0,1440.0,1530.0
195870,7.0,24.0,800.0,1027.0


In [6]:
taxi_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195872 entries, 0 to 195871
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   taxi_in   195872 non-null  float64
 1   taxi_out  195872 non-null  float64
 2   dep_time  195872 non-null  float64
 3   arr_time  195872 non-null  float64
dtypes: float64(4)
memory usage: 6.0 MB


In [ ]:
taxi_df['dep_time'] = pd.to_datetime(taxi_df['dep_time'], format ='%H:%M:%S')
taxi_df['arr_time'] = pd.to_datetime(taxi_df['arr_time'], format ='%H:%M:%S')

In [ ]:
taxi_df['dep_hour'] = taxi_df['dep_time'].dt.hour
taxi_df['arr_hour'] = taxi_df['arr_time'].dt.hour
taxi_df

In [ ]:
mean_taxi_out = taxi_df.groupby('dep_hour')['taxi_out'].mean()
mean_taxi_in = taxi_df.groupby('dep_hour')['taxi_in'].mean()

In [ ]:
fig, axes = plt.subplots(figsize=(15,10))
sns.lineplot(ax=axes, x=mean_taxi_in.index, y=mean_taxi_in.values)
plt.xticks(mean_taxi_in.index);
axes.set_title('Average Taxi-in by hour')
axes.set_xlabel('Hours')
axes.set_ylabel('Time')
plt.show()

In [ ]:
fig, axes = plt.subplots(figsize=(15,10))
sns.lineplot(ax=axes, x=mean_taxi_out.index, y=mean_taxi_out.values)
plt.xticks(mean_taxi_in.index);
axes.set_title('Average Taxi-in by hour')
axes.set_xlabel('Hours')
axes.set_ylabel('Time')
plt.show()